<a href="https://colab.research.google.com/github/NicolasNevaL/Phyton_Machine_Learning/blob/main/Modelo_BERT_Transfer_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MODELO BERT

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:

## Importamos las librerías necesarias
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
## Importamos el dataset

df = pd.read_excel("https://raw.githubusercontent.com/Izainea/nlp_ean/main/Datos/Datos%20Crudos/Noticias.xlsx")
df.head()

,Columna1,Enlaces,Título,info,contenido,Etiqueta
0,0,https://www.eltiempo.com/agresion-contra-un-op...,Operador de grúa quedó inconsciente tras agres...,El conductor de una moto le lanzó el casco y p...,Las autoridades están buscando al conductor de...,colombia
1,1,https://www.eltiempo.com/archivo/documento/CMS...,"Usaquén, primera en infracciones por mal parqueo",La localidad ocupa el primer lugar en comparen...,"""Los andenes son para los peatones"", reclama e...",archivo
2,2,https://www.eltiempo.com/archivo/documento/CMS...,'Me atracaron y vi un arma que me heló la sang...,Un ciudadano relata cómo cuatro hombres lo rob...,A las 7 de la noche me había quedado de encont...,archivo
3,3,https://www.eltiempo.com/archivo/documento/CMS...,"Escoltas mal estacionados, dolor de cabeza de ...",Las zonas de restaurantes se convierten en par...,Atravesados. Eso es lo que se les pasa por la ...,archivo
4,4,https://www.eltiempo.com/archivo/documento/CMS...,Radicado primer proyecto que autorizaría union...,"El representante de 'la U', Miguel Gómez, dijo...",“Estamos proponiendo la figura de un contrato ...,archivo


In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Pre Procesamiento

In [ ]:
## Aplicamos un preprocesamiento pequeño
df=df.dropna()

categories = ['deportes', 'cultura', 'economia', 'justicia']
df = df[df['Etiqueta'].isin(categories)]

def preprocess_text(text):
    text = text.lower()
    text = ''.join([c for c in text if c not in string.punctuation and not c.isdigit()])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('spanish')]
    return ' '.join(tokens)


df['contenido_preprocesado'] = df['contenido'].apply(preprocess_text)

In [ ]:

## Importamos el autotokenizer y el modelo

from transformers import AutoTokenizer

model_name = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name,num_labels=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
## BERT para clasificar texto

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(categories))

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
### Miremos especificaciones del modelo

model.config

BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}

In [ ]:
## Usamos el modelo

from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
## Tokenización y padding

max_len = 512
df['input_ids'] = df['contenido_preprocesado'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=max_len)
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Creamos máscaras de atención
df['attention_mask'] = df['input_ids'].apply(
    lambda x: [1] * len(x) + [0] * (max_len - len(x)) if len(x) < max_len else [1] * max_len
)

df['input_ids'] = df['input_ids'].apply(
    lambda x: x + [0] * (max_len - len(x)) if len(x) < max_len else x[:max_len]
)

In [ ]:
# Convertimos etiquetas a valores numéricos

label2id = {cat: i for i, cat in enumerate(categories)}
df['labels'] = df['Etiqueta'].map(label2id)

In [ ]:
# Separamos datos de entrenamiento y validación

X_train, X_val, y_train, y_val = train_test_split(
    df[['input_ids', 'attention_mask']],
    df['labels'],
    test_size=0.2,
    random_state=42
)

In [ ]:
# Creamos los datasets que acepta el modelo

train_dataset = Dataset.from_dict({
    'input_ids': list(X_train['input_ids']),
    'attention_mask': list(X_train['attention_mask']),
    'labels': list(y_train)
})

val_dataset = Dataset.from_dict({
    'input_ids': list(X_val['input_ids']),
    'attention_mask': list(X_val['attention_mask']),
    'labels': list(y_val)
})

In [ ]:
# Definimos las métricas a calcular

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': accuracy, 'f1': f1}

In [ ]:
# Configuramos los hiperparámetros del entrenamiento

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Entrenador del Modelo

In [ ]:
# Ejecutamos el entrenador

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entrenar el modelo
trainer.train()

<ipython-input-17-3cabfc4df07c>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.225400,0.191970,0.947802,0.947947
2,0.025500,0.278937,0.953297,0.953569
3,0.001700,0.344079,0.945055,0.945353
4,0.001000,0.269332,0.950549,0.950849
5,0.000900,0.288373,0.947802,0.948090


TrainOutput(global_step=910, training_loss=0.09451012040007409, metrics={'train_runtime': 790.047, 'train_samples_per_second': 9.215, 'train_steps_per_second': 1.152, 'total_flos': 1915482879098880.0, 'train_loss': 0.09451012040007409, 'epoch': 5.0})

In [ ]:
from sklearn.metrics import classification_report

# Función para preprocesar una noticia específica
noticia_1 = """
El Museo Nacional celebra su aniversario número 200 con una serie de exposiciones únicas que recorren la historia del arte colombiano. Las actividades incluyen talleres educativos para niños y adultos, visitas guiadas, y presentaciones de artistas locales.
"""
noticia_2 = """
el estadio el campín y su entorno se preparan para una renovación total a partir de 2025.
el escenario será completamente nuevo, para lo cual se requerirá al menos tres años de trabajo.
se espera que esté terminado en febrero de 2029.
la casa de los dos clubes tradicionales del fútbol capitalino, santa fe y millonarios, se estrenó en 1938 y tuvo tres remodelaciones en 1951, 1968 y 2011.
hoy cuenta con una capacidad de 36,343 espectadores.
el campín tendrá gramilla híbrida.
el césped natural será reemplazado por gramilla híbrida.
el estadio pascual guerrero vivió un proceso similar para el mundial femenino sub-20.
investigaciones buscan diseñar la grama ideal con respaldo de fifa y la academia.
se sembrará una variedad de raygrass, ideal para climas fríos y sombra.
se instalará un sistema de luces ultravioleta para mantener la grama.
el nuevo campín tendrá techo retráctil y aforo para 50,000 espectadores.
el proyecto está a cargo de sencia y convertirá el estadio en un escenario multipropósito.
tendrá tribuna retráctil, seis niveles de acceso y áreas vip.
protegerá la cancha de lluvias y heladas características de bogotá.
"""

def preprocess_and_tokenize(text, tokenizer, max_len):
    """
    Preprocesa y tokeniza el texto para validación con el modelo.
    """
    # Preprocesamiento
    text = text.lower()
    text = ''.join([c for c in text if c not in string.punctuation and not c.isdigit()])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('spanish')]
    preprocessed_text = ' '.join(tokens)

    # Tokenización y padding
    encoding = tokenizer(
        preprocessed_text,
        truncation=True,
        max_length=max_len,
        padding="max_length",
        return_tensors="pt"
    )
    return encoding

# Preprocesar y tokenizar la noticia de cultura
encoding = preprocess_and_tokenize(noticia_1, tokenizer, max_len)

# Move input tensors to the same device as the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Determine the device
encoding = {k: v.to(device) for k, v in encoding.items()} # Move encoding tensors to device
model.to(device) # Move the model to device if it's not already there

# Predecir la etiqueta de la noticia
with torch.no_grad():
    logits = model(
        input_ids=encoding['input_ids'],
        attention_mask=encoding['attention_mask']
    ).logits
    predicted_class = torch.argmax(logits, dim=1).item()

# Mostrar la etiqueta predicha
predicted_label = list(label2id.keys())[list(label2id.values()).index(predicted_class)]
print(f"La noticia pertenece a la categoría: {predicted_label}")


La noticia pertenece a la categoría: cultura


In [ ]:
# Generar el classification report
val_preds = []
val_labels = []

# Obtener predicciones para el conjunto de validación
for i in range(len(X_val)):
    inputs = {
        'input_ids': torch.tensor([X_val.iloc[i]['input_ids']]).to(device), # Move input tensors to device
        'attention_mask': torch.tensor([X_val.iloc[i]['attention_mask']]).to(device) # Move input tensors to device
    }
    with torch.no_grad():
        logits = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask']
        ).logits
    val_preds.append(torch.argmax(logits, dim=1).item())
    val_labels.append(y_val.iloc[i])


In [ ]:
# Reporte de clasificación
report = classification_report(val_labels, val_preds, target_names=categories)
print("\nReporte de clasificación:\n")
print(report)



Reporte de clasificación:

              precision    recall  f1-score   support

    deportes       1.00      0.96      0.98       135
     cultura       0.96      0.91      0.94        79
    economia       0.91      0.96      0.94        78
    justicia       0.91      0.97      0.94        72

    accuracy                           0.95       364
   macro avg       0.95      0.95      0.95       364
weighted avg       0.96      0.95      0.95       364



## Conclusiones
* Los modelos de transfer learning adecuados para la necesidad en cada situación facilitan la transferencia de conocimiento, ayudan a la eficiencia en el uso de los recursps (gastos de cálculos computacionales y eficiencia energética), adicionalmente, permite personalizar el modelo (Fine Tunned), ahorra tiempos en la creación de un modelo desde cero, su entrenamiento y arquitectura.

* Es importante entender muy bien los datos con los que se está trabajando, es decir el análisis exploratorio.

* Finalmente, las métricas del modelo de transfer learning fueron significativamente más adecuadas, comparadas con las métricas de los modelos RNN y LSTM.

